In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
!{sys.executable} -m pip install optuna

     |████████████████████████████████| 365 kB 10.3 MB/s 
     |████████████████████████████████| 212 kB 18.4 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3


In [3]:
import sys
!{sys.executable} -m pip install scikit-learn-intelex

     |████████████████████████████████| 83 kB 1.3 MB/s 
     |████████████████████████████████| 12.6 MB 30.5 MB/s 
     |████████████████████████████████| 85.5 MB 120 kB/s 
     |████████████████████████████████| 4.0 MB 24.0 MB/s 


In [4]:
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True)

Scikit-learn was successfully globally patched by Intel(R) Extension for Scikit-learn


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('https://gitlab.com/mirsakhawathossain/exodata/-/raw/main/dataset/exodata.csv').sort_index(axis=0)

In [7]:
columns_1 = df.columns[df.mean()==1]
columns_2 = df.columns[df.mean()==0]
df = df.drop(columns=columns_1)
df = df.drop(columns=columns_2)

In [8]:
X = df.drop(['exoplanet'],axis=1)

In [9]:
X.shape

(75458, 741)

In [10]:
y = df[['exoplanet']]

In [11]:
y.value_counts()

exoplanet
1            40084
0            35374
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=43,stratify=y)

In [14]:
print(X_train.shape)
print(X_test.shape)

(45274, 741)
(30184, 741)


In [15]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer()

In [16]:
X_train = qt.fit_transform(X_train)
X_test = qt.transform(X_test)

In [17]:
import optuna
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.tree import DecisionTreeClassifier


In [18]:
def objective(trial):
    # Define the hyperparameter search space
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])


In [19]:
def objective(trial):
    # Define the hyperparameter search space
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    # Train and evaluate the decision tree classifier with given hyperparameters
    clf = DecisionTreeClassifier(max_depth=max_depth,
                                  min_samples_split=min_samples_split,
                                  min_samples_leaf=min_samples_leaf,
                                  max_features=max_features,
                                  criterion=criterion,
                                  random_state=43)
    
    rskf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=41)
    accs = cross_val_score(clf, X_train, y_train, cv=rskf)
    return 1 - np.mean(accs)

In [20]:
# Define the study object to manage the optimization process
study = optuna.create_study(direction='minimize')

[I 2023-03-16 07:48:54,343] A new study created in memory with name: no-name-1f9d152b-f7c3-405b-adc6-241f2aff9bda


In [21]:
# Run the hyperparameter optimization
n_trials = 50
n_jobs = -1
study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)

[I 2023-03-16 07:50:53,260] Trial 13 finished with value: 0.42828336563842906 and parameters: {'max_depth': 2, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 13 with value: 0.42828336563842906.
[I 2023-03-16 07:51:03,084] Trial 28 finished with value: 0.3288641935734511 and parameters: {'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 'log2', 'criterion': 'gini'}. Best is trial 28 with value: 0.3288641935734511.
[I 2023-03-16 07:51:03,332] Trial 31 finished with value: 0.3800834626573951 and parameters: {'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 28 with value: 0.3288641935734511.
[I 2023-03-16 07:51:04,968] Trial 17 finished with value: 0.32881560163529533 and parameters: {'max_depth': 5, 'min_samples_split': 7, 'min_samples_leaf': 8, 'max_features': 'log2', 'criterion': 'gini'}. Best is trial 17 with value: 0.3

In [22]:
# Print the best hyperparameters found
print('Best hyperparameters:')
print(study.best_params)

Best hyperparameters:
{'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 10, 'max_features': None, 'criterion': 'gini'}
